In [1]:
import numpy as np
from keras.models import Model,Sequential,load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.optimizers import *
from keras import backend as K
K.set_image_data_format('channels_last')

Using CNTK backend


In [2]:
train_data_dir = 'data/train'
validation_data_dir = 'data/validate'
epochs = 100
batch_size = 16
image_size = 30
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   rotation_range=30)
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    seed = 1997)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    seed = 1997)


Found 129658 images belonging to 88 classes.
Found 25448 images belonging to 88 classes.


In [3]:
def create_model(size):
    model = Sequential()
    init = 'he_normal'
    model.add(Conv2D(32, (2, 2), activation = 'relu', name = 'block1_conv1', 
                            kernel_initializer = init, input_shape = (size, size, 3)))
    model.add(Conv2D(32, (2, 2), activation = 'relu', name = 'block1_conv2', kernel_initializer = init))
    model.add(MaxPooling2D((2, 2), name = 'block1_pool'))

    model.add(Conv2D(64, (2, 2), activation = 'relu', name = 'block2_conv1', kernel_initializer = init))
    model.add(Conv2D(128, (2, 2), activation = 'relu', name = 'block2_conv2', kernel_initializer = init))
    model.add(MaxPooling2D((2, 2), name = 'block2_pool'))

    model.add(Conv2D(256, (2, 2), activation = 'relu', name = 'block3_conv1', kernel_initializer = init))
    model.add(Conv2D(512, (2, 2), activation = 'relu', name = 'block3_conv2', kernel_initializer = init))
    model.add(MaxPooling2D((2, 2), name = 'block3_pool'))
    
#     model.add(Conv2D(512, (2, 2), activation = 'relu', name = 'block4_conv1', kernel_initializer=init))
#     model.add(Conv2D(512, (2, 2), activation = 'relu', name = 'block4_conv2', kernel_initializer=init))
#     model.add(MaxPooling2D((2, 2), name = 'block4_pool'))
    
    model.add(Flatten())
    model.add(Dense(512, activation = 'relu', kernel_initializer = init, name = 'fc1'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation = 'relu', kernel_initializer = init, name = 'fc2'))
    model.add(Dropout(0.5))
    model.add(Dense(88, activation = 'softmax', name = 'predictions'))
    
    optim = Adam(lr = 0.0005)
    
    model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])
    return model

model = create_model(size = image_size)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 29, 29, 32)        416       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 28, 28, 32)        4128      
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 14, 14, 32)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 13, 13, 64)        8256      
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 12, 12, 128)       32896     
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 6, 6, 128)         0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 5, 5, 256)         131328    
__________

In [4]:
model.fit_generator(
        train_generator,
        steps_per_epoch = 8000,
        epochs = epochs,
        validation_data = validation_generator,
        validation_steps = 1600)

Epoch 1/100
8000/8000 [==============================] - 2337s - loss: 4.1840 - acc: 0.0411 - val_loss: 3.6917 - val_acc: 0.1012
Epoch 2/100
8000/8000 [==============================] - 353s - loss: 3.7305 - acc: 0.1046 - val_loss: 3.3579 - val_acc: 0.1699
Epoch 3/100
8000/8000 [==============================] - 333s - loss: 3.5060 - acc: 0.1464 - val_loss: 3.1061 - val_acc: 0.2228
Epoch 4/100
8000/8000 [==============================] - 333s - loss: 3.3645 - acc: 0.1763 - val_loss: 2.9393 - val_acc: 0.2668
Epoch 5/100
8000/8000 [==============================] - 332s - loss: 3.2635 - acc: 0.1987 - val_loss: 2.8567 - val_acc: 0.2904
Epoch 6/100
8000/8000 [==============================] - 333s - loss: 3.1792 - acc: 0.2168 - val_loss: 2.7338 - val_acc: 0.3206
Epoch 7/100
8000/8000 [==============================] - 334s - loss: 3.1191 - acc: 0.2318 - val_loss: 2.7028 - val_acc: 0.3197
Epoch 8/100
8000/8000 [==============================] - 334s - loss: 3.0628 - acc: 0.2455 - val_loss: 

8000/8000 [==============================] - 339s - loss: 2.5083 - acc: 0.3946 - val_loss: 1.9992 - val_acc: 0.5250
Epoch 65/100
8000/8000 [==============================] - 339s - loss: 2.5087 - acc: 0.3957 - val_loss: 1.9993 - val_acc: 0.5162
Epoch 66/100
8000/8000 [==============================] - 339s - loss: 2.5086 - acc: 0.3950 - val_loss: 2.0149 - val_acc: 0.5198
Epoch 67/100
8000/8000 [==============================] - 339s - loss: 2.5029 - acc: 0.3955 - val_loss: 2.0166 - val_acc: 0.5141
Epoch 68/100
8000/8000 [==============================] - 340s - loss: 2.5003 - acc: 0.3979 - val_loss: 2.0503 - val_acc: 0.5064
Epoch 69/100
8000/8000 [==============================] - 339s - loss: 2.5023 - acc: 0.3965 - val_loss: 2.0440 - val_acc: 0.5057
Epoch 70/100
8000/8000 [==============================] - 339s - loss: 2.4996 - acc: 0.3990 - val_loss: 1.9771 - val_acc: 0.5243
Epoch 71/100
8000/8000 [==============================] - 339s - loss: 2.5130 - acc: 0.3964 - val_loss: 2.0008

In [5]:
model.save_weights('model.h5') 